In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import librosa    
import os

from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from sklearn.model_selection import train_test_split as tts

In [76]:
###### Open train and test file
# If first time
#train = pd.read_csv("train.csv")
#test = pd.read_csv("test.csv")

# If data ready
train = pd.read_csv("train_data_fft.csv")
test = pd.read_csv("test_data_fft.csv")

In [77]:
train = train.drop(["Unnamed: 0"], axis=1)
test = test.drop(["Unnamed: 0"], axis=1)

In [78]:
train.head()

,0,1,2,3,4,5,6,7,8,9,...,8184,8185,8186,8187,8188,8189,8190,8191,ID,Class
0,0.010143,0.001146,0.000447,0.000879,0.002113,0.001961,0.001428,0.001496,0.001667,0.001677,...,0.048349,0.033911,0.020420,0.013966,0.003319,0.011668,0.057645,0.028970,0,siren
1,0.036623,0.011112,0.063523,0.034905,0.028611,0.066069,0.044936,0.005443,0.038347,0.029738,...,0.038273,0.040522,0.028949,0.013680,0.005235,0.037265,0.033057,0.016678,1,street_music
2,0.376092,0.020247,0.023632,0.042894,0.049975,0.015479,0.047714,0.079284,0.074942,0.030555,...,0.289756,0.134134,0.413688,0.399814,0.148875,0.193186,0.239007,0.364078,2,drilling
3,0.000351,0.001262,0.000465,0.000391,0.000630,0.001142,0.000765,0.001085,0.000967,0.000978,...,0.000573,0.001663,0.001531,0.001734,0.001155,0.000791,0.001518,0.001946,3,siren
4,0.053220,0.009434,0.036519,0.044301,0.074437,0.142190,0.223753,0.062588,0.073502,0.181442,...,0.007472,0.011931,0.008514,0.008406,0.011658,0.012671,0.012001,0.009600,4,dog_bark


In [79]:
train["Class"].unique()

array(['siren', 'street_music', 'drilling', 'dog_bark',
       'children_playing', 'gun_shot', 'engine_idling', 'air_conditioner',
       'jackhammer', 'car_horn'], dtype=object)

In [80]:
encoder = LabelEncoder()
train["Class"] = encoder.fit_transform(train["Class"])

In [116]:
encoder.classes_

array(['air_conditioner', 'car_horn', 'children_playing', 'dog_bark',
       'drilling', 'engine_idling', 'gun_shot', 'jackhammer', 'siren',
       'street_music'], dtype=object)

In [81]:
train.head()

,0,1,2,3,4,5,6,7,8,9,...,8184,8185,8186,8187,8188,8189,8190,8191,ID,Class
0,0.010143,0.001146,0.000447,0.000879,0.002113,0.001961,0.001428,0.001496,0.001667,0.001677,...,0.048349,0.033911,0.020420,0.013966,0.003319,0.011668,0.057645,0.028970,0,8
1,0.036623,0.011112,0.063523,0.034905,0.028611,0.066069,0.044936,0.005443,0.038347,0.029738,...,0.038273,0.040522,0.028949,0.013680,0.005235,0.037265,0.033057,0.016678,1,9
2,0.376092,0.020247,0.023632,0.042894,0.049975,0.015479,0.047714,0.079284,0.074942,0.030555,...,0.289756,0.134134,0.413688,0.399814,0.148875,0.193186,0.239007,0.364078,2,4
3,0.000351,0.001262,0.000465,0.000391,0.000630,0.001142,0.000765,0.001085,0.000967,0.000978,...,0.000573,0.001663,0.001531,0.001734,0.001155,0.000791,0.001518,0.001946,3,8
4,0.053220,0.009434,0.036519,0.044301,0.074437,0.142190,0.223753,0.062588,0.073502,0.181442,...,0.007472,0.011931,0.008514,0.008406,0.011658,0.012671,0.012001,0.009600,4,3


In [82]:
bitrate = 16384
columns = []
columns_str = []

for x in range(bitrate//2):
    columns.append(x)
    columns_str.append(str(x))

In [6]:
os.chdir("Train/")

temp = pd.DataFrame(data=None, columns=columns)
temp["ID"] = ""
i = 0
for filename in os.listdir():   
    i += 1
    file, rate = librosa.load(filename, sr=bitrate)
    fft = np.fft.fft(file, norm="ortho")
    
    #### Normalize fft
    result = abs(fft[:rate//2])
    result *= (1/result.max())
    
    if len(result) < rate//2:
        diff = rate//2 - len(result)
        result = np.append(result, np.zeros(diff))
    
    #plt.plot(file)
    #plt.show()
    #plt.plot(result)
    #plt.show()
    
    name, _ = filename.split(".")
    a = dict((a, result[a]) for a in columns)
    a["ID"] = name
    temp.loc[name] = pd.Series(a)
    
    if i%100==0:
        print(i/len(train["ID"])*100,"%")
        
os.chdir("..")
temp["ID"] = temp["ID"].astype('int64')
temp.sort_values(by=[8192])
train.reset_index(drop=True, inplace=True)
temp.reset_index(drop=True, inplace=True)
train = pd.concat([temp, train], axis=1, ignore_index=True)

## temporary fix to misnamed columns
train = train.drop([8193], axis=1)
train = train.rename(columns={8192:"ID", 8494:"Class"})
    
train.to_csv("train_data_fft.csv", index=False)

1.8399264029438822 %
3.6798528058877644 %
5.519779208831647 %
7.359705611775529 %
9.19963201471941 %
11.039558417663294 %
12.879484820607177 %
14.719411223551058 %
16.55933762649494 %
18.39926402943882 %
20.239190432382706 %
22.079116835326587 %
23.91904323827047 %
25.758969641214353 %
27.598896044158234 %
29.438822447102115 %
31.278748850046 %
33.11867525298988 %
34.95860165593376 %
36.79852805887764 %
38.638454461821524 %
40.47838086476541 %
42.318307267709294 %
44.158233670653175 %
45.998160073597056 %
47.83808647654094 %
49.678012879484825 %
51.517939282428706 %
53.35786568537259 %
55.19779208831647 %
57.03771849126035 %
58.87764489420423 %
60.71757129714811 %
62.557497700092 %
64.39742410303589 %
66.23735050597976 %
68.07727690892365 %
69.91720331186752 %
71.75712971481141 %
73.59705611775529 %
75.43698252069917 %
77.27690892364305 %
79.11683532658694 %
80.95676172953083 %
82.7966881324747 %
84.63661453541859 %
86.47654093836246 %
88.31646734130635 %
90.15639374425022 %
91.9963201

In [51]:
os.chdir("Test/")
temp = pd.DataFrame(data=None, columns=columns)
temp["ID"] = ""
i = 0

for filename in os.listdir():   
    i += 1
    file, rate = librosa.load(filename, sr=bitrate)
    fft = np.fft.fft(file, norm="ortho")
    
    #### Normalize fft
    result = abs(fft[:rate//2])
    result *= (1/result.max())
    
    if len(result) < rate//2:
        diff = rate//2 - len(result)
        result = np.append(result, np.zeros(diff))
    
    #plt.plot(file)
    #plt.show()
    #plt.plot(result)
    #plt.show()
    
    name, _ = filename.split(".")
    a = dict((a, result[a]) for a in columns)
    a["ID"] = name
    temp.loc[name] = pd.Series(a)
    
    if i%100==0:
        print(i/len(test["ID"])*100,"%")
    
os.chdir("..")
temp["ID"] = temp["ID"].astype('int64')
temp = temp.sort_values(by=['ID'])
test.reset_index(drop=True, inplace=True)
temp.reset_index(drop=True, inplace=True)
test = pd.concat([temp, test], axis=1, ignore_index=True)

## temporary fix to misnamed columns
test = test.drop([8193], axis=1)
test = test.rename(columns={8192:"ID"})
    
test.to_csv("test_data_fft.csv", index=False)

3.0330603579011224 %
6.066120715802245 %
9.099181073703367 %
12.13224143160449 %
15.16530178950561 %
18.198362147406733 %
21.231422505307858 %
24.26448286320898 %
27.2975432211101 %
30.33060357901122 %
33.363663936912346 %
36.39672429481347 %
39.42978465271459 %
42.462845010615716 %
45.49590536851683 %
48.52896572641796 %
51.56202608431908 %
54.5950864422202 %
57.62814680012133 %
60.66120715802244 %
63.69426751592356 %
66.72732787382469 %
69.76038823172581 %
72.79344858962693 %
75.82650894752805 %
78.85956930542918 %
81.8926296633303 %
84.92569002123143 %
87.95875037913254 %
90.99181073703366 %
94.0248710949348 %
97.05793145283592 %


KeyError: 8192

In [8]:
test

,0,1,2,3,4,5,6,7,8,9,...,8183,8184,8185,8186,8187,8188,8189,8190,8191,ID
0,0.044005,0.003445,0.007560,0.049354,0.033493,0.039656,0.031924,0.009465,0.005210,0.030861,...,0.077500,0.113409,0.156148,0.083681,0.032360,0.095703,0.078591,0.056351,0.170996,5
1,0.036184,0.003229,0.003987,0.004174,0.002115,0.002848,0.002616,0.002013,0.002581,0.001474,...,0.006089,0.015891,0.001911,0.018957,0.014953,0.006702,0.009144,0.014492,0.017143,7
2,0.007616,0.003814,0.004545,0.004256,0.003073,0.002439,0.004204,0.003266,0.002082,0.003162,...,0.023224,0.037052,0.043134,0.029996,0.030960,0.053059,0.016577,0.050036,0.075980,8
3,0.001557,0.000977,0.000939,0.000816,0.000875,0.001008,0.000901,0.000817,0.001107,0.001017,...,0.014341,0.006255,0.021703,0.016837,0.015881,0.015046,0.013412,0.033381,0.004233,9
4,0.354227,0.001966,0.002432,0.003261,0.004804,0.001349,0.002336,0.002222,0.002554,0.004227,...,0.009932,0.009575,0.025866,0.003951,0.007912,0.012189,0.019164,0.010006,0.007654,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3292,0.001555,0.003564,0.002988,0.002946,0.000975,0.000559,0.002332,0.001081,0.002240,0.003324,...,0.269776,0.193090,0.235372,0.106630,0.120741,0.244023,0.268525,0.282726,0.157627,8708
3293,0.113239,0.238018,0.480341,0.125074,0.086868,0.166905,0.202994,0.260916,0.269623,0.228712,...,0.008069,0.008708,0.008292,0.007200,0.006507,0.006498,0.006332,0.006728,0.007362,8718
3294,0.054025,0.000599,0.004454,0.000998,0.003044,0.002216,0.001393,0.004586,0.002748,0.005824,...,0.031863,0.032705,0.008153,0.005888,0.043145,0.038221,0.019212,0.021848,0.040741,8719
3295,0.005392,0.006314,0.009456,0.007219,0.034041,0.047491,0.037096,0.084044,0.040924,0.053079,...,0.073604,0.085196,0.044669,0.039594,0.049665,0.004077,0.123424,0.096649,0.101070,8730


In [9]:
X_train, X_test, y_train, y_test = tts(train[columns_str], train["Class"], test_size=0.3, random_state=42)

In [83]:
train_copy = train

In [84]:
target = train_copy.pop('Class')
train_copy = train_copy.drop(["ID"], axis=1)
dataset = tf.data.Dataset.from_tensor_slices((train_copy.values, target.values))

In [85]:
for feat, targ in dataset.take(5):
    print ('Features: {}, Target: {}'.format(feat, targ))

Features: [0.01014347 0.00114603 0.00044739 ... 0.01166771 0.05764549 0.02897034], Target: 8
Features: [0.03662339 0.01111194 0.06352294 ... 0.03726491 0.03305687 0.01667803], Target: 9
Features: [0.37609202 0.02024659 0.02363217 ... 0.19318632 0.23900674 0.36407785], Target: 4
Features: [0.0003514  0.00126206 0.00046481 ... 0.00079056 0.00151757 0.00194614], Target: 8
Features: [0.05321973 0.00943366 0.03651887 ... 0.01267124 0.01200143 0.00960041], Target: 3


In [86]:
train_dataset = dataset.shuffle(len(train)).batch(1)

In [87]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
32(0.2)/32(0.2)//50 epochs 50.08%/~28%

In [91]:
model.fit(train_dataset, epochs=50)

Epoch 1/50
5435/5435 [==============================] - 13s 2ms/step - loss: 1.7659 - accuracy: 0.3454
Epoch 2/50
5435/5435 [==============================] - 7s 1ms/step - loss: 1.7381 - accuracy: 0.3492
Epoch 3/50
5435/5435 [==============================] - 7s 1ms/step - loss: 1.7139 - accuracy: 0.3636
Epoch 4/50
5435/5435 [==============================] - 7s 1ms/step - loss: 1.6862 - accuracy: 0.3794
Epoch 5/50
5435/5435 [==============================] - 7s 1ms/step - loss: 1.6664 - accuracy: 0.3856
Epoch 6/50
5435/5435 [==============================] - 7s 1ms/step - loss: 1.6448 - accuracy: 0.3823
Epoch 7/50
5435/5435 [==============================] - 7s 1ms/step - loss: 1.6330 - accuracy: 0.3869
Epoch 8/50
5435/5435 [==============================] - 7s 1ms/step - loss: 1.6291 - accuracy: 0.3873
Epoch 9/50
5435/5435 [==============================] - 7s 1ms/step - loss: 1.6024 - accuracy: 0.3983
Epoch 10/50
5435/5435 [==============================] - 6s 1ms/step - loss: 1.57

In [117]:
######## Save output ##########
result = model.predict(test[columns_str])

out = pd.read_csv("sample_submission_kopia.csv")
out['ID'] = test['ID']
out['Class'] = encoder.inverse_transform(np.argmax(result, axis=1))

out.to_csv('sample_submission.csv', index=False)